In [1]:
%pip install pymongo


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pymongo import MongoClient
import pandas as pd
import json

In [9]:
keys = {}
with open('keys.json', 'r') as file:
    keys = json.load(file)

In [6]:
file_path = keys['taxonomyResFile']
print(file_path)

./data/taxonomy_path.csv


In [7]:
taxonomy_path_df = pd.read_csv(file_path)

In [8]:
def convert_to_list(data):
    data = data[2:-2]
    if data:
        return data.split("', '")  # Assuming items are separated by commas
    else:
        return []

new_taxonomy_path = taxonomy_path_df['taxonomy_path'].apply(convert_to_list)
new_taxonomy_rank = taxonomy_path_df['taxonomy_rank'].apply(convert_to_list)

In [22]:
new_taxonomy_path.loc[0]

['root', 'cellular organisms', 'Bacteria']

In [23]:
taxonomy_path_df['taxonomy_path'] = new_taxonomy_path
taxonomy_path_df['taxonomy_rank'] = new_taxonomy_rank

In [24]:
taxonomy_path_df

,id,taxonomy_path,taxonomy_rank
0,2,"[root, cellular organisms, Bacteria]","[0, no rank, superkingdom]"
1,6,"[root, cellular organisms, Bacteria, Pseudomon...","[0, no rank, superkingdom, phylum, class, orde..."
2,7,"[root, cellular organisms, Bacteria, Pseudomon...","[0, no rank, superkingdom, phylum, class, orde..."
3,9,"[root, cellular organisms, Bacteria, Pseudomon...","[0, no rank, superkingdom, phylum, class, orde..."
4,10,"[root, cellular organisms, Bacteria, Pseudomon...","[0, no rank, superkingdom, phylum, class, orde..."
...,...,...,...
2520227,3070616,"[root, Viruses, Duplodnaviria, Heunggongvirae,...","[0, superkingdom, clade, kingdom, phylum, clas..."
2520228,3070820,"[root, Viruses, Duplodnaviria, Heunggongvirae,...","[0, superkingdom, clade, kingdom, phylum, clas..."
2520229,3070859,"[root, cellular organisms, Eukaryota, Sar, Str...","[0, no rank, superkingdom, clade, clade, phylu..."
2520230,3070949,"[root, cellular organisms, Bacteria, Pseudomon...","[0, no rank, superkingdom, phylum, class, orde..."


In [25]:
def createNewCollection(db, collectionName):
    # Create the new collection
    
    collection = db[collectionName]

    print(f"Collection '{collectionName}' created successfully.")

    return collection

In [26]:
# Set MongoDB connection uri
connectionURI = keys['connectionURI']


Collection 'taxonomy' created successfully.


In [ ]:

# Connect to MongoDB
client = MongoClient(connectionURI)

# Select the database and collection
db = client.tumorBoard
collection = createNewCollection(db, "taxonomy")

In [27]:
data_to_insert = taxonomy_path_df.to_dict(orient="records")

In [28]:
data_to_insert[:5]

[{'id': 2,
  'taxonomy_path': ['root', 'cellular organisms', 'Bacteria'],
  'taxonomy_rank': ['0', 'no rank', 'superkingdom']},
 {'id': 6,
  'taxonomy_path': ['root',
   'cellular organisms',
   'Bacteria',
   'Pseudomonadota',
   'Alphaproteobacteria',
   'Hyphomicrobiales',
   'Xanthobacteraceae',
   'Azorhizobium'],
  'taxonomy_rank': ['0',
   'no rank',
   'superkingdom',
   'phylum',
   'class',
   'order',
   'family',
   'genus']},
 {'id': 7,
  'taxonomy_path': ['root',
   'cellular organisms',
   'Bacteria',
   'Pseudomonadota',
   'Alphaproteobacteria',
   'Hyphomicrobiales',
   'Xanthobacteraceae',
   'Azorhizobium',
   'Azorhizobium caulinodans'],
  'taxonomy_rank': ['0',
   'no rank',
   'superkingdom',
   'phylum',
   'class',
   'order',
   'family',
   'genus',
   'species']},
 {'id': 9,
  'taxonomy_path': ['root',
   'cellular organisms',
   'Bacteria',
   'Pseudomonadota',
   'Gammaproteobacteria',
   'Enterobacterales',
   'Erwiniaceae',
   'Buchnera',
   'Buchnera ap

In [29]:
# Delete all documents in the collection
result = collection.delete_many({})

# Print the number of deleted documents
print(f'Deleted {result.deleted_count} documents')

Deleted 2520232 documents


In [30]:
collection.insert_many(data_to_insert)

In [31]:
client.close()